In [ ]:
import tensorflow as tf

# Training + Validation split (80/20)
train_ds = tf.keras.utils.image_dataset_from_directory(
    r"D:\DataSET\Training",
    image_size=(512, 512),
    batch_size=16,
    validation_split=0.2,   # 20% for validation
    subset="training",
    seed=123
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    r"D:\DataSET\Training",
    image_size=(512, 512),
    batch_size=16,
    validation_split=0.2,
    subset="validation",
    seed=123
)

# Testing dataset
test_ds = tf.keras.utils.image_dataset_from_directory(
    r"D:\DataSET\Testing",
    image_size=(512, 512),
    batch_size=16
)

# Normalize pixel values
train_ds = train_ds.map(lambda x, y: (x/255.0, y))
val_ds   = val_ds.map(lambda x, y: (x/255.0, y))
test_ds  = test_ds.map(lambda x, y: (x/255.0, y))


In [ ]:
#Pure LSTM Model : Model1

In [ ]:
from tensorflow.keras import layers, models

# Pure LSTM model
def build_lstm_model(input_shape=(512,512,3), num_classes=4):
    inputs = layers.Input(shape=input_shape)
    x = layers.Reshape((512, 512*3))(inputs)   # treat each row as a timestep
    x = layers.LSTM(256, return_sequences=True)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.LSTM(128)(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs, name="LSTM_only")
    return model

# Build and compile
model1 = build_lstm_model()
model1.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

# Train & evaluate
history1 = model1.fit(train_ds, epochs=10, validation_data=val_ds, verbose=1)
loss1, acc1 = model1.evaluate(test_ds, verbose=1)
print(f"Model1 (LSTM-only) Test Accuracy: {acc1:.2f}")

In [ ]:
#Pure CBAM Model : Model2

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# CBAM block
def cbam_block(feature_map, ratio=8):
    channel = feature_map.shape[-1]

    # Channel Attention
    shared_dense_one = layers.Dense(channel // ratio, activation='relu')
    shared_dense_two = layers.Dense(channel)

    avg_pool = layers.GlobalAveragePooling2D()(feature_map)
    avg_pool = layers.Reshape((1,1,channel))(avg_pool)
    avg_pool = shared_dense_two(shared_dense_one(avg_pool))

    max_pool = layers.GlobalMaxPooling2D()(feature_map)
    max_pool = layers.Reshape((1,1,channel))(max_pool)
    max_pool = shared_dense_two(shared_dense_one(max_pool))

    channel_attention = layers.Activation('sigmoid')(avg_pool + max_pool)
    channel_refined = layers.multiply([feature_map, channel_attention])

    # Spatial Attention
    avg_pool = layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(channel_refined)
    max_pool = layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(channel_refined)
    concat = layers.Concatenate(axis=-1)([avg_pool, max_pool])
    spatial_attention = layers.Conv2D(1, kernel_size=7, padding='same', activation='sigmoid')(concat)

    refined_feature = layers.multiply([channel_refined, spatial_attention])
    return refined_feature

# Deep CBAM branch with proper residuals
def build_cbam_branch(input_shape=(512,512,3)):
    inputs = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2,2))(x)
    x = cbam_block(x)

    # Block 2 with residual projection
    skip = x
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = cbam_block(x)
    skip = layers.Conv2D(64, (1,1), strides=(2,2), padding='same')(skip)
    x = layers.Add()([x, skip])

    # Block 3 with residual projection
    skip = x
    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = cbam_block(x)
    skip = layers.Conv2D(128, (1,1), strides=(2,2), padding='same')(skip)
    x = layers.Add()([x, skip])

    # Block 4 (extra depth, no residual)
    x = layers.Conv2D(256, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = cbam_block(x)

    # Global pooling
    x = layers.GlobalAveragePooling2D()(x)

    return models.Model(inputs, x, name="Deep_CBAM_branch")

In [ ]:
model2 = build_cbam_branch(input_shape=(512,512,3))
outputs2 = layers.Dense(4, activation='softmax')(model2.output)
model2 = models.Model(model2.input, outputs2, name="Deep_CBAM")

model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history2 = model2.fit(train_ds, epochs=10, validation_data=val_ds, verbose=1)
loss2, acc2 = model2.evaluate(test_ds, verbose=1)
print(f"Model2 (Deep CBAM) Test Accuracy: {acc2:.2f}")

In [ ]:
#Final LSTM + CBAM : Model3

In [ ]:
# Remove the final softmax layer from each model
lstm_feature_extractor = models.Model(
    inputs=model1.input,
    outputs=model1.layers[-2].output,  # penultimate layer
    name="LSTM_features"
)

cbam_feature_extractor = models.Model(
    inputs=model2.input,
    outputs=model2.layers[-2].output,  # penultimate layer
    name="CBAM_features"
)

In [ ]:
# Shared input
inputs = layers.Input(shape=(512,512,3))

# Get features from both models
lstm_features = lstm_feature_extractor(inputs)
cbam_features = cbam_feature_extractor(inputs)

# Concatenate feature vectors
concat = layers.Concatenate()([lstm_features, cbam_features])

# Classification head
x = layers.Dense(128, activation='relu')(concat)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(4, activation='softmax')(x)

# Final hybrid model
model3 = models.Model(inputs=inputs, outputs=outputs, name="Hybrid_LSTM_CBAM")

# Compile
model3.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

model3.summary()

In [ ]:
history3 = model3.fit(train_ds, epochs=10, validation_data=val_ds, verbose=1)
loss3, acc3 = model3.evaluate(test_ds, verbose=1)
print(f"Model3 (Hybrid LSTM+CBAM) Test Accuracy: {acc3:.2f}")